In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB2 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

C:\Users\parkh\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


--------------------------------------------------
flow time: 26562, util : 0.958, makespan : 849
n_epi: 0, score : -4844.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 27256, util : 0.945, makespan : 869
n_episode: 1, score : -4945.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 26562, util : 0.958, makespan : 849
n_epi: 1, score : -4844.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 27526, util : 0.944, makespan : 839
n_episode: 2, score : -4931.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 26562, util : 0.958, makespan : 849
n_epi: 2, score : -4844.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 27009, util : 0.948, makespan : 875
n_episode: 3, score : -4841.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 26562, util

<ipython-input-2-5c70b09296d6>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 28235, util : 0.947, makespan : 868
n_episode: 5, score : -4919.0, n_buffer : 2040, eps : 8.0%
--------------------------------------------------
flow time: 26826, util : 0.955, makespan : 849
n_epi: 5, score : -4865.0, n_buffer : 2040, eps : 0.0%
--------------------------------------------------
flow time: 27147, util : 0.943, makespan : 837
n_episode: 6, score : -4879.0, n_buffer : 2380, eps : 7.9%
--------------------------------------------------
flow time: 29157, util : 0.949, makespan : 865
n_epi: 6, score : -4913.0, n_buffer : 2380, eps : 0.0%
--------------------------------------------------
flow time: 29835, util : 0.942, makespan : 835
n_episode: 7, score : -4919.0, n_buffer : 2720, eps : 7.9%
--------------------------------------------------
flow time: 31013, util : 0.946, makespan : 854
n_epi: 7, score : -4925.0, n_buffer : 2720, eps : 0.0%
--------------------------------------------------
flow time: 30794, u

--------------------------------------------------
flow time: 27984, util : 0.923, makespan : 814
n_epi: 31, score : -4589.0, n_buffer : 10880, eps : 0.0%
--------------------------------------------------
flow time: 29015, util : 0.923, makespan : 819
n_episode: 32, score : -4651.0, n_buffer : 11220, eps : 7.7%
--------------------------------------------------
flow time: 28029, util : 0.924, makespan : 765
n_epi: 32, score : -4407.0, n_buffer : 11220, eps : 0.0%
--------------------------------------------------
flow time: 28792, util : 0.908, makespan : 765
n_episode: 33, score : -4469.0, n_buffer : 11560, eps : 7.7%
--------------------------------------------------
flow time: 28367, util : 0.916, makespan : 801
n_epi: 33, score : -4600.0, n_buffer : 11560, eps : 0.0%
--------------------------------------------------
flow time: 29208, util : 0.914, makespan : 802
n_episode: 34, score : -4611.0, n_buffer : 11900, eps : 7.7%
--------------------------------------------------
flow ti

...

--------------------------------------------------
flow time: 25517, util : 0.902, makespan : 775
n_episode: 241, score : -4183.0, n_buffer : 82280, eps : 5.6%
--------------------------------------------------
flow time: 33189, util : 0.896, makespan : 732
n_epi: 241, score : -4329.0, n_buffer : 82280, eps : 0.0%
--------------------------------------------------
flow time: 34154, util : 0.882, makespan : 766
n_episode: 242, score : -4495.0, n_buffer : 82620, eps : 5.6%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 242, score : -3986.0, n_buffer : 82620, eps : 0.0%
--------------------------------------------------
flow time: 24409, util : 0.925, makespan : 747
n_episode: 243, score : -4116.0, n_buffer : 82960, eps : 5.6%
--------------------------------------------------
flow time: 25553, util : 0.932, makespan : 764
n_epi: 243, score : -4407.0, n_buffer : 82960, eps : 0.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 27722, util : 0.905, makespan : 731
n_episode: 267, score : -4195.0, n_buffer : 91120, eps : 5.3%
--------------------------------------------------
flow time: 24328, util : 0.932, makespan : 732
n_epi: 267, score : -4075.0, n_buffer : 91120, eps : 0.0%
--------------------------------------------------
flow time: 25403, util : 0.925, makespan : 698
n_episode: 268, score : -4087.0, n_buffer : 91460, eps : 5.3%
--------------------------------------------------
flow time: 24509, util : 0.922, makespan : 725
n_epi: 268, score : -4083.0, n_buffer : 91460, eps : 0.0%
--------------------------------------------------
flow time: 24219, util : 0.911, makespan : 722
n_episode: 269, score : -4118.0, n_buffer : 91800, eps : 5.3%
--------------------------------------------------
flow time: 23967, util : 0.924, makespan : 718
n_epi: 269, score : -4097.0, n_buffer : 91800, eps : 0.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 25890, util : 0.924, makespan : 716
n_episode: 293, score : -4077.0, n_buffer : 99960, eps : 5.1%
--------------------------------------------------
flow time: 23957, util : 0.931, makespan : 721
n_epi: 293, score : -4031.0, n_buffer : 99960, eps : 0.0%
--------------------------------------------------
flow time: 24842, util : 0.913, makespan : 694
n_episode: 294, score : -4113.0, n_buffer : 100000, eps : 5.1%
--------------------------------------------------
flow time: 24736, util : 0.910, makespan : 743
n_epi: 294, score : -4084.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25356, util : 0.903, makespan : 712
n_episode: 295, score : -4091.0, n_buffer : 100000, eps : 5.1%
--------------------------------------------------
flow time: 28476, util : 0.887, makespan : 707
n_epi: 295, score : -4162.0, n_buffer : 100000, eps : 0.0%
------------------------------------------------

--------------------------------------------------
flow time: 25862, util : 0.904, makespan : 678
n_episode: 319, score : -4017.0, n_buffer : 100000, eps : 4.8%
--------------------------------------------------
flow time: 23646, util : 0.924, makespan : 719
n_epi: 319, score : -4055.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24930, util : 0.922, makespan : 692
n_episode: 320, score : -4043.0, n_buffer : 100000, eps : 4.8%
--------------------------------------------------
flow time: 24285, util : 0.920, makespan : 734
n_epi: 320, score : -4105.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24436, util : 0.923, makespan : 725
n_episode: 321, score : -4082.0, n_buffer : 100000, eps : 4.8%
--------------------------------------------------
flow time: 33516, util : 0.889, makespan : 707
n_epi: 321, score : -4175.0, n_buffer : 100000, eps : 0.0%
----------------------------------------------

--------------------------------------------------
flow time: 25675, util : 0.899, makespan : 712
n_episode: 345, score : -4140.0, n_buffer : 100000, eps : 4.5%
--------------------------------------------------
flow time: 24832, util : 0.929, makespan : 736
n_epi: 345, score : -4047.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 26147, util : 0.912, makespan : 716
n_episode: 346, score : -4116.0, n_buffer : 100000, eps : 4.5%
--------------------------------------------------
flow time: 24186, util : 0.928, makespan : 732
n_epi: 346, score : -4016.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25000, util : 0.917, makespan : 748
n_episode: 347, score : -4108.0, n_buffer : 100000, eps : 4.5%
--------------------------------------------------
flow time: 27582, util : 0.892, makespan : 669
n_epi: 347, score : -3965.0, n_buffer : 100000, eps : 0.0%
----------------------------------------------

In [1]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB2 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    params = torch.load("362q.pt")
    q.load_state_dict(params)
    q.eval()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(362, 1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 23753, util : 0.936, makespan : 726
n_episode: 362, score : -4039.0, n_buffer : 340, eps : 4.4%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 362, score : -3986.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 24969, util : 0.926, makespan : 746
n_episode: 363, score : -4130.0, n_buffer : 680, eps : 4.4%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 363, score : -3986.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 25272, util : 0.925, makespan : 732
n_episode: 364, score : -4062.0, n_buffer : 1020, eps : 4.4%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 364, score : -3986.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 

<ipython-input-1-938e23fd0e2d>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 23564, util : 0.929, makespan : 740
n_episode: 367, score : -4118.0, n_buffer : 2040, eps : 4.3%
--------------------------------------------------
flow time: 38972, util : 0.867, makespan : 764
n_epi: 367, score : -4520.0, n_buffer : 2040, eps : 0.0%
--------------------------------------------------
flow time: 38931, util : 0.874, makespan : 797
n_episode: 368, score : -4721.0, n_buffer : 2380, eps : 4.3%
--------------------------------------------------
flow time: 25244, util : 0.923, makespan : 704
n_epi: 368, score : -4093.0, n_buffer : 2380, eps : 0.0%
--------------------------------------------------
flow time: 26377, util : 0.908, makespan : 735
n_episode: 369, score : -4177.0, n_buffer : 2720, eps : 4.3%
--------------------------------------------------
flow time: 34450, util : 0.888, makespan : 672
n_epi: 369, score : -3941.0, n_buffer : 2720, eps : 0.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 25342, util : 0.889, makespan : 761
n_epi: 393, score : -4291.0, n_buffer : 10880, eps : 0.0%
--------------------------------------------------
flow time: 25280, util : 0.902, makespan : 749
n_episode: 394, score : -4149.0, n_buffer : 11220, eps : 4.1%
--------------------------------------------------
flow time: 24389, util : 0.935, makespan : 727
n_epi: 394, score : -4065.0, n_buffer : 11220, eps : 0.0%
--------------------------------------------------
flow time: 24370, util : 0.926, makespan : 765
n_episode: 395, score : -4117.0, n_buffer : 11560, eps : 4.0%
--------------------------------------------------
flow time: 24052, util : 0.945, makespan : 737
n_epi: 395, score : -4012.0, n_buffer : 11560, eps : 0.0%
--------------------------------------------------
flow time: 25296, util : 0.916, makespan : 729
n_episode: 396, score : -4082.0, n_buffer : 11900, eps : 4.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 24505, util : 0.908, makespan : 757
n_epi: 419, score : -4119.0, n_buffer : 19720, eps : 0.0%
--------------------------------------------------
flow time: 25894, util : 0.895, makespan : 694
n_episode: 420, score : -4108.0, n_buffer : 20060, eps : 3.8%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 420, score : -3986.0, n_buffer : 20060, eps : 0.0%
--------------------------------------------------
flow time: 25076, util : 0.930, makespan : 743
n_episode: 421, score : -4087.0, n_buffer : 20400, eps : 3.8%
--------------------------------------------------
flow time: 24554, util : 0.931, makespan : 715
n_epi: 421, score : -4049.0, n_buffer : 20400, eps : 0.0%
--------------------------------------------------
flow time: 25061, util : 0.927, makespan : 715
n_episode: 422, score : -4084.0, n_buffer : 20740, eps : 3.8%
--------------------------------------------------
f

...

--------------------------------------------------
flow time: 24061, util : 0.925, makespan : 712
n_epi: 887, score : -4036.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24095, util : 0.923, makespan : 738
n_episode: 888, score : -4007.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_epi: 888, score : -3986.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24112, util : 0.937, makespan : 740
n_episode: 889, score : -4059.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24399, util : 0.914, makespan : 718
n_epi: 889, score : -4094.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24552, util : 0.912, makespan : 728
n_episode: 890, score : -4131.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 23842, util : 0.946, makespan : 729
n_epi: 913, score : -4022.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23382, util : 0.938, makespan : 725
n_episode: 914, score : -4012.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24130, util : 0.928, makespan : 729
n_epi: 914, score : -4072.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23580, util : 0.923, makespan : 727
n_episode: 915, score : -4070.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24243, util : 0.920, makespan : 755
n_epi: 915, score : -4063.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24246, util : 0.918, makespan : 749
n_episode: 916, score : -4060.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 23278, util : 0.933, makespan : 738
n_epi: 939, score : -3996.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23350, util : 0.935, makespan : 738
n_episode: 940, score : -4010.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24583, util : 0.917, makespan : 740
n_epi: 940, score : -4069.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24510, util : 0.919, makespan : 728
n_episode: 941, score : -4065.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 25931, util : 0.894, makespan : 756
n_epi: 941, score : -4142.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25982, util : 0.887, makespan : 729
n_episode: 942, score : -4176.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 23887, util : 0.939, makespan : 731
n_epi: 965, score : -3993.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23787, util : 0.944, makespan : 687
n_episode: 966, score : -3998.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23195, util : 0.937, makespan : 728
n_epi: 966, score : -4023.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23195, util : 0.937, makespan : 728
n_episode: 967, score : -4023.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24244, util : 0.920, makespan : 731
n_epi: 967, score : -4079.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 24346, util : 0.914, makespan : 730
n_episode: 968, score : -4078.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------

--------------------------------------------------
flow time: 23641, util : 0.932, makespan : 739
n_epi: 991, score : -4036.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23641, util : 0.932, makespan : 739
n_episode: 992, score : -4036.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23731, util : 0.931, makespan : 715
n_epi: 992, score : -4026.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 23831, util : 0.927, makespan : 716
n_episode: 993, score : -4057.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24098, util : 0.917, makespan : 717
n_epi: 993, score : -4032.0, n_buffer : 100000, eps : 0.0%
--------------------------------------------------
flow time: 25146, util : 0.902, makespan : 744
n_episode: 994, score : -4158.0, n_buffer : 100000, eps : 1.0%
----------------------------------------------